In [ ]:
import argparse
import csv
import logging
import re
import requests
import xml.etree.ElementTree as ET
from typing import List, Dict, Optional

# Configure logging
logging.basicConfig(level=logging.INFO)

def fetch_pubmed_ids(query: str) -> List[str]:
    """Fetch PubMed IDs for a given query."""
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {"db": "pubmed", "term": query, "retmode": "xml", "retmax": 100}
    response = requests.get(url, params=params)
    if response.status_code != 200:
        logging.error("Failed to fetch PubMed IDs")
        return []
    root = ET.fromstring(response.text)
    return [id_elem.text for id_elem in root.findall(".//Id")]

def fetch_pubmed_details(pubmed_id: str) -> Optional[Dict]:
    """Fetch details of a PubMed article."""
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {"db": "pubmed", "id": pubmed_id, "retmode": "xml"}
    response = requests.get(url, params=params)
    if response.status_code != 200:
        logging.error(f"Failed to fetch details for PubMed ID: {pubmed_id}")
        return None
    root = ET.fromstring(response.text)
    
    title = root.find(".//ArticleTitle")
    date = root.find(".//PubDate/Year")
    authors = []
    companies = []
    email = ""
    
    for author in root.findall(".//Author"):
        affiliation = author.find("AffiliationInfo/Affiliation")
        if affiliation is not None and re.search(r"(Inc\.|Ltd\.|Pharma|Biotech)", affiliation.text, re.I):
            authors.append(author.find("LastName").text)
            companies.append(affiliation.text)
        if author.find("Email") is not None:
            email = author.find("Email").text
    
    return {
        "PubmedID": pubmed_id,
        "Title": title.text if title is not None else "",
        "Publication Date": date.text if date is not None else "",
        "Non-academic Author(s)": ", ".join(authors),
        "Company Affiliation(s)": ", ".join(companies),
        "Corresponding Author Email": email,
    }

def save_to_csv(data: List[Dict], filename: str):
    """Save fetched data to CSV."""
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)
    logging.info(f"Results saved to {filename}")

def main():
    parser = argparse.ArgumentParser(description="Fetch research papers from PubMed.")
    parser.add_argument("query", type=str, help="PubMed search query")
    parser.add_argument("-f", "--file", type=str, help="Output CSV filename", default="output.csv")
    parser.add_argument("-d", "--debug", action="store_true", help="Enable debug mode")
    
    args = parser.parse_args()
    
    if args.debug:
        logging.getLogger().setLevel(logging.DEBUG)
    
    pubmed_ids = fetch_pubmed_ids(args.query)
    results = [fetch_pubmed_details(pid) for pid in pubmed_ids if pid]
    results = [r for r in results if r is not None]
    
    if args.file:
        save_to_csv(results, args.file)
    else:
        print(results)

if __name__ == "__main__":
    main()
